In [1]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import plot_tree


#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')
from ml_pipeline import train_n_predict, validation, clean_split
from Data.data_cleaner import cleaner

In [2]:
## Clean and split data
train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed.csv"
clean_data = cleaner(train_path, feature_path, morph_path)

In [3]:
clean_data

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,AL_pre,V1_pre,RL_post,V1_post,AL_post,area1,area2,area3,minicol_dist,nuclei_adp_dist
0,42593,0.445791,-1.742979,2.174696,0.444810,-1.740997,2.176315,-1.515697,2.007164,0.931055,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.076496
1,42594,0.507204,0.365932,0.044530,0.507420,0.368718,0.052049,-1.225578,1.070301,0.268030,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.306546
2,42595,0.460781,-1.807635,2.215744,0.449166,-1.817435,2.210853,0.632301,2.099967,0.905442,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.118612
3,42596,0.433668,-1.686017,2.114604,0.442015,-1.682527,2.140593,0.645635,1.931656,0.956222,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.035778
4,42597,0.451199,0.293911,0.745737,0.449891,0.328971,0.733044,1.334344,1.668055,-0.042977,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.587712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185827,228420,0.709191,-0.639091,0.441756,0.717718,-0.655646,0.436799,0.452526,-0.282486,0.324338,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.214541,-0.780397
185828,228421,0.543500,-1.794867,0.474081,0.537437,-1.818791,0.460571,0.687832,-0.042144,0.817518,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.047859
185829,228422,0.577400,-2.030573,-0.272713,0.566982,-2.048065,-0.249929,0.987837,0.182085,0.910021,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.385538
185830,228423,0.684728,-1.930889,-0.146313,0.698481,-1.913531,-0.140918,1.162151,-0.079219,0.792450,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.261090


In [4]:
import random

def splitter(data):
# perform stratified sampling of pre-synaptic neurons
    pre_nucleus_ids = pd.unique(data["pre_nucleus_id"])
    #print(len(pre_nucleus_ids))

    # Use 80% of the pre-nucleus ids
    #print(len(pre_nucleus_ids))
    train_nucleus_idx = random.sample(range(0, len(pre_nucleus_ids)), int(np.floor(0.8*len(pre_nucleus_ids))))
    train_nucleus_ids = pre_nucleus_ids[train_nucleus_idx]
    training = data[data["pre_nucleus_id"].isin(train_nucleus_ids)]
    X_train = training.drop(columns='connected')
    y_train = training['connected']
    pre_nucleus_ids = np.delete(pre_nucleus_ids, train_nucleus_idx)

    # # Use 20% for query set
    # #print(len(pre_nucleus_ids))
    # query_nucleus_idx = random.sample(range(0, len(pre_nucleus_ids)), int(np.floor(0.5*len(pre_nucleus_ids))))
    # query_nucleus_ids = pre_nucleus_ids[query_nucleus_idx]
    # query = data[data["pre_nucleus_id"].isin(query_nucleus_ids)]
    # X_query = query.drop(columns='connected')
    # y_query = query['connected']
    # pre_nucleus_ids = np.delete(pre_nucleus_ids, query_nucleus_idx)


    # Use 20% for validation
    #print(len(pre_nucleus_ids))
    validation = data[data["pre_nucleus_id"].isin(pre_nucleus_ids)]
    X_val = validation.drop(columns='connected')
    y_val = validation['connected']

    return X_train, X_val, y_train, y_val

In [5]:
random.seed(1)
X_train, X_val, y_train, y_val = splitter(clean_data)
X_train

,ID,axonal_coor_x,axonal_coor_y,axonal_coor_z,dendritic_coor_x,dendritic_coor_y,dendritic_coor_z,adp_dist,post_skeletal_distance_to_soma,pre_skeletal_distance_to_soma,...,AL_pre,V1_pre,RL_post,V1_post,AL_post,area1,area2,area3,minicol_dist,nuclei_adp_dist
0,42593,0.445791,-1.742979,2.174696,0.444810,-1.740997,2.176315,-1.515697,2.007164,0.931055,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.076496
1,42594,0.507204,0.365932,0.044530,0.507420,0.368718,0.052049,-1.225578,1.070301,0.268030,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.306546
2,42595,0.460781,-1.807635,2.215744,0.449166,-1.817435,2.210853,0.632301,2.099967,0.905442,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.118612
3,42596,0.433668,-1.686017,2.114604,0.442015,-1.682527,2.140593,0.645635,1.931656,0.956222,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,0.035778
4,42597,0.451199,0.293911,0.745737,0.449891,0.328971,0.733044,1.334344,1.668055,-0.042977,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.262374,-0.587712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185827,228420,0.709191,-0.639091,0.441756,0.717718,-0.655646,0.436799,0.452526,-0.282486,0.324338,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,1.166722,-0.202758,-0.214541,-0.780397
185828,228421,0.543500,-1.794867,0.474081,0.537437,-1.818791,0.460571,0.687832,-0.042144,0.817518,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.047859
185829,228422,0.577400,-2.030573,-0.272713,0.566982,-2.048065,-0.249929,0.987837,0.182085,0.910021,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.385538
185830,228423,0.684728,-1.930889,-0.146313,0.698481,-1.913531,-0.140918,1.162151,-0.079219,0.792450,...,-0.545885,-0.813463,1.182234,-0.852369,-0.439881,-1.076967,-0.857102,4.931999,-0.214541,0.261090


In [6]:
X_cv = X_train.copy()
X_cv['connected']=y_train

In [7]:
# n_leaves = range(2, 17, 2)
max_depths = range(3, 15)
eta_val = np.arange(0.1, 1.1, 0.1)
lambda_val = np.arange(0, 2.1, 0.1)

accuracies = {}
for depth in max_depths:
    for lam in lambda_val:
        for etav in eta_val:
            fold_accuracy = 0
            for fold in [1,2,3,4,5]:      
                #Making  fold
                random.seed(fold)
                X_train_fold, X_val_fold, y_train_fold, y_val_fold = splitter(X_cv)
                ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
                X_train_fold, y_train_fold = ros.fit_resample(
                    X_train_fold, y_train_fold)  
                X_train_fold = X_train_fold.select_dtypes('number')
                X_val_fold = X_val_fold.select_dtypes('number')    
                
                XGB = XGBClassifier(max_depth=depth, reg_lambda=lam, eta=etav) 
                # print(np.shape(X_train_fold), np.shape(X_val_fold))

                XGB.fit(X_train_fold.drop(columns = ["ID","pre_nucleus_id","post_nucleus_id"]),y_train_fold)
                y_hat_valid = XGB.predict(X_val_fold.drop(columns = ["ID","pre_nucleus_id","post_nucleus_id"]))
                valid_acc = balanced_accuracy_score(y_val_fold, y_hat_valid)
                fold_accuracy += valid_acc
                print(f"tree depth: {depth}, lambda: {lam}, eta: {etav} valid accuracy for this fold, {valid_acc}")
            avg_fold_accuracy = fold_accuracy/5
            print(f"avgfold accuracy: {avg_fold_accuracy}")
            accuracies[(depth, lam, etav)] = avg_fold_accuracy
    

tree depth: 3, lambda: 0.0, eta: 0.1 valid accuracy for this fold, 0.7623287129743324
tree depth: 3, lambda: 0.0, eta: 0.1 valid accuracy for this fold, 0.7531517380463612
tree depth: 3, lambda: 0.0, eta: 0.1 valid accuracy for this fold, 0.7638414802887465
tree depth: 3, lambda: 0.0, eta: 0.1 valid accuracy for this fold, 0.7573709167058738
tree depth: 3, lambda: 0.0, eta: 0.1 valid accuracy for this fold, 0.760873655088829
avgfold accuracy: 0.7595133006208286
tree depth: 3, lambda: 0.0, eta: 0.2 valid accuracy for this fold, 0.7325291111638161
tree depth: 3, lambda: 0.0, eta: 0.2 valid accuracy for this fold, 0.7050465001652015
tree depth: 3, lambda: 0.0, eta: 0.2 valid accuracy for this fold, 0.7434212556883781
tree depth: 3, lambda: 0.0, eta: 0.2 valid accuracy for this fold, 0.7112551776307899
tree depth: 3, lambda: 0.0, eta: 0.2 valid accuracy for this fold, 0.6989678369927064
avgfold accuracy: 0.7182439763281785
tree depth: 3, lambda: 0.0, eta: 0.30000000000000004 valid accuracy

In [19]:
accuracy_df = pd.DataFrame({'Param Pair':accuracies.keys(), 'Avg Accuracy':accuracies.values()}).sort_values(by='Avg Accuracy', ascending=False).reset_index()
accuracy_df

,index,Param Pair,Avg Accuracy
0,120,"(3, 1.2000000000000002, 0.1)",0.767577
1,140,"(3, 1.4000000000000001, 0.1)",0.764933
2,130,"(3, 1.3, 0.1)",0.764705
3,200,"(3, 2.0, 0.1)",0.763951
4,190,"(3, 1.9000000000000001, 0.1)",0.763810
...,...,...,...
2515,1442,"(9, 1.8, 0.30000000000000004)",0.499655
2516,1125,"(8, 0.7000000000000001, 0.6)",0.499642
2517,1638,"(10, 1.6, 0.9)",0.499573
2518,2098,"(12, 2.0, 0.9)",0.499573


In [9]:
accuracy_df.to_csv('xgb_accuracy_est_leaf_eta_cv.csv')

In [10]:
best_param = accuracy_df['Param Pair'][0]
best_param

(3, 1.2000000000000002, 0.1)

In [11]:
X_train=X_train.select_dtypes('number')
ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
X_train_use, y_train_use = ros.fit_resample(
                    X_train, y_train) 
X_val = X_val.select_dtypes('number')

In [12]:
xg_model = XGBClassifier(max_depth=best_param[0], reg_lambda=best_param[1], eta=best_param[2])
xg_model.fit(X_train_use.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1), y_train_use)

preds = xg_model.predict(X_val.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))
balanced_accuracy_score(y_val, preds)

0.7514618564046632

In [13]:
best_features = pd.DataFrame({"Features":X_train_use.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).columns, 
              "Importances":abs(xg_model.feature_importances_)}).sort_values(by='Importances', ascending=False)
best_features

,Features,Importances
6,adp_dist,0.198647
14,post_test_score,0.050794
26,RL_pre,0.045855
8,pre_skeletal_distance_to_soma,0.043878
7,post_skeletal_distance_to_soma,0.043502
32,area1,0.034427
29,RL_post,0.034214
18,pre_nucleus_y,0.033688
15,post_rf_x,0.031976
19,pre_nucleus_z,0.031256


In [14]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)

In [15]:
sub_data = sub_data.select_dtypes('number')
sub_data.columns

Index(['ID', 'axonal_coor_x', 'axonal_coor_y', 'axonal_coor_z',
       'dendritic_coor_x', 'dendritic_coor_y', 'dendritic_coor_z', 'adp_dist',
       'post_skeletal_distance_to_soma', 'pre_skeletal_distance_to_soma',
       'pre_oracle', 'pre_test_score', 'pre_rf_x', 'pre_rf_y', 'post_oracle',
       'post_test_score', 'post_rf_x', 'post_rf_y', 'pre_nucleus_x',
       'pre_nucleus_y', 'pre_nucleus_z', 'post_nucleus_x', 'post_nucleus_y',
       'post_nucleus_z', 'pre_nucleus_id', 'post_nucleus_id', 'me_similarity',
       'fw_similarity', 'rf_similarity', 'AL_pre', 'RL_pre', 'V1_pre',
       'V1_post', 'RL_post', 'AL_post', 'area1', 'area2', 'area3',
       'minicol_dist', 'nuclei_adp_dist'],
      dtype='object')

In [16]:
preds = xg_model.predict(sub_data.drop(columns=['ID', 'pre_nucleus_id', 'post_nucleus_id']).sort_index(axis=1))


In [17]:
sub_data['connected'] = preds==1

In [18]:
submission_data = sub_data.filter(['ID','connected'])
submission_data.to_csv('submission_data_xg.csv',index=False)